# 📚 Comprendre les données SAE (Statistique Annuelle des Établissements)

Ce notebook explique la structure et le contenu des fichiers SAE utilisés dans ce projet.

**Source:** DREES (Direction de la Recherche, des Études, de l'Évaluation et des Statistiques)  
**Données:** 2019 à 2024  
**Établissement cible:** Pitié-Salpêtrière (FINESS: 750100125)

## 1. Qu'est-ce que la SAE ?

La **Statistique Annuelle des Établissements de santé (SAE)** est une enquête administrative exhaustive, collectée chaque année par la DREES.

### Caractéristiques:
- **Exhaustive**: Couvre TOUS les établissements de santé en France
- **Obligatoire**: Chaque établissement doit déclarer ses données
- **Annuelle**: Collecte au 1er semestre de l'année N+1 pour les données de l'année N

### Contenu:
- Capacités d'accueil (lits, places)
- Activité (séjours, journées, passages)
- Équipements
- Personnel

In [1]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path('../data')
FINESS_PITIE = '750100125'

print("✅ Configuration chargée")

✅ Configuration chargée


## 2. Structure des fichiers SAE

Chaque année contient environ **50-54 fichiers CSV**, organisés par thématique.

In [2]:
# Liste des fichiers SAE par catégorie
SAE_MODULES = {
    '🏥 Identification': {
        'ID': 'Identification de l\'établissement (FINESS, nom, adresse, catégorie)',
        'GHT': 'Groupement Hospitalier de Territoire',
        'ORG': 'Organisation générale',
    },
    '🛏️ Capacités MCO': {
        'MCO': 'Médecine-Chirurgie-Obstétrique (lits, places, séjours, journées)',
        'MCOPED': 'MCO Pédiatrie',
        'MCOGER': 'MCO Gériatrie',
        'MCOAVC': 'MCO - Unités neurovasculaires (AVC)',
        'MCOSURV': 'MCO Surveillance continue',
    },
    '🚨 Soins Critiques': {
        'REA': 'Réanimation, Soins Intensifs, USC (lits, journées, séjours par type unité)',
    },
    '🚑 Urgences': {
        'URGENCES': 'Autorisations et organisation des urgences',
        'URGENCES2': 'Activité des urgences (passages, hospitalisations, durées)',
        'SMURSAMU': 'SMUR/SAMU (sorties, régulation)',
    },
    '🧠 Psychiatrie': {
        'PSY': 'Psychiatrie (lits, places, files actives, actes)',
        'PSY2': 'Psychiatrie détaillée',
    },
    '🏋️ SSR': {
        'SSR': 'Soins de Suite et Réadaptation (lits, places, séjours)',
        'SSR2': 'SSR détaillé par spécialité',
    },
    '🏠 HAD': {
        'HAD': 'Hospitalisation à Domicile',
    },
    '👶 Périnatalité': {
        'PERINAT': 'Maternité (accouchements, lits obstétrique/néonatologie)',
        'SYGEN': 'Gynécologie',
    },
    '🔧 Équipements': {
        'BLOCS': 'Blocs opératoires (salles, postes SSPI)',
        'IMAGES': 'Imagerie (scanners, IRM, gamma-caméras)',
    },
    '💊 Activités spécialisées': {
        'CANCERO': 'Cancérologie (chimiothérapie, radiothérapie)',
        'DIALYSE': 'Dialyse',
        'CHIRCAR': 'Chirurgie cardiaque',
        'NEUROCHIR': 'Neurochirurgie',
        'BRULES': 'Grands brûlés',
    },
    '👥 Personnel': {
        'PN': 'Personnel non médical (ETP par catégorie)',
        '*_P': 'Fichiers _P = Personnel par service (ex: REA_P, BLOCS_P)',
    },
}

print("📁 STRUCTURE DES FICHIERS SAE")
print("="*60)
for category, modules in SAE_MODULES.items():
    print(f"\n{category}")
    for code, desc in modules.items():
        print(f"   {code:12} → {desc}")

📁 STRUCTURE DES FICHIERS SAE

🏥 Identification
   ID           → Identification de l'établissement (FINESS, nom, adresse, catégorie)
   GHT          → Groupement Hospitalier de Territoire
   ORG          → Organisation générale

🛏️ Capacités MCO
   MCO          → Médecine-Chirurgie-Obstétrique (lits, places, séjours, journées)
   MCOPED       → MCO Pédiatrie
   MCOGER       → MCO Gériatrie
   MCOAVC       → MCO - Unités neurovasculaires (AVC)
   MCOSURV      → MCO Surveillance continue

🚨 Soins Critiques
   REA          → Réanimation, Soins Intensifs, USC (lits, journées, séjours par type unité)

🚑 Urgences
   URGENCES     → Autorisations et organisation des urgences
   URGENCES2    → Activité des urgences (passages, hospitalisations, durées)
   SMURSAMU     → SMUR/SAMU (sorties, régulation)

🧠 Psychiatrie
   PSY          → Psychiatrie (lits, places, files actives, actes)
   PSY2         → Psychiatrie détaillée

🏋️ SSR
   SSR          → Soins de Suite et Réadaptation (lits, places, séj

## 3. Colonnes clés par fichier

In [3]:
# Colonnes communes à tous les fichiers
COLONNES_COMMUNES = {
    'BOR': 'Identifiant du bordereau (ex: MCO, REA)',
    'AN': 'Année des données',
    'FI': 'FINESS de l\'établissement (identifiant unique)',
    'RS': 'Raison sociale (nom de l\'établissement)',
    'FI_EJ': 'FINESS de l\'entité juridique (ex: AP-HP)',
}

# Colonnes spécifiques MCO
COLONNES_MCO = {
    'LIT_MED': 'Lits médecine (hospitalisation complète)',
    'LIT_CHI': 'Lits chirurgie',
    'LIT_OBS': 'Lits obstétrique',
    'LIT_MCO': 'Total lits MCO',
    'PLA_MED': 'Places ambulatoire médecine',
    'PLA_CHI': 'Places ambulatoire chirurgie',
    'PLA_MCO': 'Total places ambulatoire',
    'SEJHC_MED': 'Séjours hospitalisation complète médecine',
    'SEJHC_CHI': 'Séjours hospitalisation complète chirurgie',
    'SEJHP_MED': 'Séjours hospitalisation partielle médecine',
    'JOU_MED': 'Journées médecine',
    'JOU_CHI': 'Journées chirurgie',
    'JLI_MED': 'Journées-lits disponibles médecine',
}

# Colonnes REA
COLONNES_REA = {
    'UNI': 'Type d\'unité (REAADU=Réa adulte, SIADU=SI adulte, USICADU=USC cardio...)',
    'LIT': 'Nombre de lits dans l\'unité',
    'JLI': 'Journées-lits disponibles',
    'SEJ': 'Nombre de séjours',
    'JOU': 'Journées réalisées',
    'PAS': 'Passages (entrées)',
}

print("📋 COLONNES COMMUNES (tous fichiers):")
for col, desc in COLONNES_COMMUNES.items():
    print(f"   {col:10} → {desc}")

print("\n📋 COLONNES MCO:")
for col, desc in list(COLONNES_MCO.items())[:8]:
    print(f"   {col:12} → {desc}")
    
print("\n📋 COLONNES REA:")
for col, desc in COLONNES_REA.items():
    print(f"   {col:10} → {desc}")

📋 COLONNES COMMUNES (tous fichiers):
   BOR        → Identifiant du bordereau (ex: MCO, REA)
   AN         → Année des données
   FI         → FINESS de l'établissement (identifiant unique)
   RS         → Raison sociale (nom de l'établissement)
   FI_EJ      → FINESS de l'entité juridique (ex: AP-HP)

📋 COLONNES MCO:
   LIT_MED      → Lits médecine (hospitalisation complète)
   LIT_CHI      → Lits chirurgie
   LIT_OBS      → Lits obstétrique
   LIT_MCO      → Total lits MCO
   PLA_MED      → Places ambulatoire médecine
   PLA_CHI      → Places ambulatoire chirurgie
   PLA_MCO      → Total places ambulatoire
   SEJHC_MED    → Séjours hospitalisation complète médecine

📋 COLONNES REA:
   UNI        → Type d'unité (REAADU=Réa adulte, SIADU=SI adulte, USICADU=USC cardio...)
   LIT        → Nombre de lits dans l'unité
   JLI        → Journées-lits disponibles
   SEJ        → Nombre de séjours
   JOU        → Journées réalisées
   PAS        → Passages (entrées)


## 4. Exemple concret: Lire les données Pitié-Salpêtrière

In [4]:
def load_pitie_sae(year, module):
    """Charge les données SAE de Pitié pour une année et un module."""
    base_dir = DATA_DIR / f"Bases CSV {year}"
    
    for suffix in ['r', 'a']:
        filepath = base_dir / f"{module}_{year}{suffix}.csv"
        if filepath.exists():
            df = pd.read_csv(filepath, sep=';', encoding='latin-1', low_memory=False)
            df['FI'] = df['FI'].astype(str)
            pitie = df[df['FI'] == FINESS_PITIE]
            return pitie
    return None

# Exemple: Charger MCO 2024
mco_2024 = load_pitie_sae(2024, 'MCO')

if mco_2024 is not None and not mco_2024.empty:
    print("📊 DONNÉES MCO 2024 - PITIÉ-SALPÊTRIÈRE")
    print("="*50)
    print(f"\n🏥 Établissement: {mco_2024['RS'].values[0]}")
    print(f"\n📊 Capacités:")
    print(f"   Lits Médecine:    {int(mco_2024['LIT_MED'].values[0])}")
    print(f"   Lits Chirurgie:   {int(mco_2024['LIT_CHI'].values[0])}")
    print(f"   Lits Obstétrique: {int(mco_2024['LIT_OBS'].values[0])}")
    print(f"   TOTAL MCO:        {int(mco_2024['LIT_MCO'].values[0])}")
    print(f"\n   Places ambulatoire: {int(mco_2024['PLA_MCO'].values[0])}")
    print(f"\n📈 Activité annuelle:")
    print(f"   Séjours médecine:  {int(mco_2024['SEJHC_MED'].values[0]):,}")
    print(f"   Séjours chirurgie: {int(mco_2024['SEJHC_CHI'].values[0]):,}")
    print(f"   Journées totales:  {int(mco_2024['JOU_MED'].values[0] + mco_2024['JOU_CHI'].values[0]):,}")
else:
    print("⚠️ Fichier MCO 2024 non trouvé")

📊 DONNÉES MCO 2024 - PITIÉ-SALPÊTRIÈRE

🏥 Établissement: GHU APHP SUN SITE PITIE SALPETRIERE

📊 Capacités:
   Lits Médecine:    732
   Lits Chirurgie:   390
   Lits Obstétrique: 46
   TOTAL MCO:        1168

   Places ambulatoire: 292

📈 Activité annuelle:
   Séjours médecine:  38,495
   Séjours chirurgie: 17,406
   Journées totales:  344,522


In [14]:
# Exemple: Réanimation
rea_2024 = load_pitie_sae(2024, 'REA')

if rea_2024 is not None and not rea_2024.empty:
    print("🚨 DONNÉES RÉANIMATION 2024 - PITIÉ-SALPÊTRIÈRE")
    print("="*50)
    print(f"\n{'Unité':<15} {'Lits':>8} {'Journées':>10} {'Séjours':>10}")
    print("-"*45)
    
    total_lits = 0
    for _, row in rea_2024.iterrows():
        uni = row.get('UNI', 'N/A')
        lits = int(row.get('LIT', 0)) if pd.notna(row.get('LIT')) else 0
        jou = int(row.get('JOU', 0)) if pd.notna(row.get('JOU')) else 0
        sej = int(row.get('SEJ', 0)) if pd.notna(row.get('SEJ')) else 0
        print(f"{uni:<15} {lits:>8} {jou:>10,} {sej:>10,}")
        total_lits += lits
    
    print("-"*45)
    print(f"{'TOTAL':<15} {total_lits:>8}")

🚨 DONNÉES RÉANIMATION 2024 - PITIÉ-SALPÊTRIÈRE

Unité               Lits   Journées    Séjours
---------------------------------------------
AUTSIADU               8      7,723      2,072
REAADU               104     36,060      4,918
SIADU                120     36,553      8,532
SITOT                120     36,553      8,532
USICADU               31     11,821      4,162
USIHADU               38     10,313        593
USINVADU              16      4,645        916
USIPADU               27      2,051        789
---------------------------------------------
TOTAL                464


## 5. Types d'unités de Réanimation (UNI)

Le fichier REA contient **plusieurs lignes par établissement**, une par type d'unité.

In [6]:
TYPES_UNITES_REA = {
    'REAADU': 'Réanimation adulte',
    'REAPED': 'Réanimation pédiatrique',
    'REANEO': 'Réanimation néonatale',
    'SIADU': 'Soins intensifs adulte (total)',
    'SITOT': 'Soins intensifs total',
    'USICADU': 'USC cardiologie adulte',
    'USIHADU': 'USC hospitalisation adulte',
    'USINVADU': 'USC neurovasculaire adulte',
    'USIPADU': 'USC pneumologie adulte',
    'AUTSIADU': 'Autres SI adulte',
}

print("📋 TYPES D'UNITÉS DE SOINS CRITIQUES:")
print("="*50)
for code, desc in TYPES_UNITES_REA.items():
    print(f"   {code:12} → {desc}")

📋 TYPES D'UNITÉS DE SOINS CRITIQUES:
   REAADU       → Réanimation adulte
   REAPED       → Réanimation pédiatrique
   REANEO       → Réanimation néonatale
   SIADU        → Soins intensifs adulte (total)
   SITOT        → Soins intensifs total
   USICADU      → USC cardiologie adulte
   USIHADU      → USC hospitalisation adulte
   USINVADU     → USC neurovasculaire adulte
   USIPADU      → USC pneumologie adulte
   AUTSIADU     → Autres SI adulte


## 6. Calcul du taux d'occupation

Le taux d'occupation se calcule ainsi:

$$\text{Taux d'occupation} = \frac{\text{Journées réalisées}}{\text{Lits} \times 365} \times 100$$

In [7]:
# Calcul taux d'occupation MCO
if mco_2024 is not None and not mco_2024.empty:
    lits = int(mco_2024['LIT_MCO'].values[0])
    journees = int(mco_2024['JOU_MED'].values[0]) + int(mco_2024['JOU_CHI'].values[0]) + int(mco_2024['JOU_OBS'].values[0])
    
    taux = (journees / (lits * 365)) * 100
    
    print("📊 CALCUL TAUX D'OCCUPATION MCO 2024")
    print("="*50)
    print(f"   Lits MCO:              {lits:,}")
    print(f"   Journées-lits dispo:   {lits * 365:,} (lits × 365)")
    print(f"   Journées réalisées:    {journees:,}")
    print(f"")
    print(f"   📈 TAUX D'OCCUPATION:  {taux:.1f}%")
    print(f"")
    print(f"   Interprétation:")
    if taux > 90:
        print(f"   ⚠️ Très élevé - Risque de tension")
    elif taux > 80:
        print(f"   ✅ Normal pour un CHU")
    else:
        print(f"   📉 Sous-utilisation potentielle")

📊 CALCUL TAUX D'OCCUPATION MCO 2024
   Lits MCO:              1,168
   Journées-lits dispo:   426,320 (lits × 365)
   Journées réalisées:    359,599

   📈 TAUX D'OCCUPATION:  84.3%

   Interprétation:
   ✅ Normal pour un CHU


## 7. Évolution des fichiers SAE (2019-2024)

Les fichiers SAE évoluent légèrement chaque année. Voici les changements notables.

In [8]:
# Comparer le nombre de fichiers par année
import os

years_files = {}
for year in [2019, 2020, 2021, 2022, 2023, 2024]:
    dir_path = DATA_DIR / f"Bases CSV {year}"
    if dir_path.exists():
        files = list(dir_path.glob("*.csv"))
        years_files[year] = {
            'count': len(files),
            'files': [f.stem for f in files]
        }

print("📁 NOMBRE DE FICHIERS SAE PAR ANNÉE:")
print("="*40)
for year, data in years_files.items():
    print(f"   {year}: {data['count']} fichiers")

📁 NOMBRE DE FICHIERS SAE PAR ANNÉE:
   2019: 53 fichiers
   2020: 49 fichiers
   2021: 53 fichiers
   2022: 53 fichiers
   2023: 53 fichiers
   2024: 54 fichiers


In [9]:
# Vérifier la présence de Pitié dans chaque année
print("\n🔍 PRÉSENCE PITIÉ-SALPÊTRIÈRE PAR ANNÉE:")
print("="*50)

for year in [2019, 2020, 2021, 2022, 2023, 2024]:
    mco = load_pitie_sae(year, 'MCO')
    if mco is not None and not mco.empty:
        lits = int(mco['LIT_MCO'].values[0]) if 'LIT_MCO' in mco.columns else 0
        print(f"   {year}: ✅ Présent - {lits} lits MCO")
    else:
        print(f"   {year}: ❌ Non trouvé")


🔍 PRÉSENCE PITIÉ-SALPÊTRIÈRE PAR ANNÉE:
   2019: ✅ Présent - 1449 lits MCO
   2020: ✅ Présent - 1447 lits MCO
   2021: ✅ Présent - 1451 lits MCO
   2022: ✅ Présent - 1219 lits MCO
   2023: ✅ Présent - 1175 lits MCO
   2024: ✅ Présent - 1168 lits MCO


## 8. Résumé: Comment utiliser ces données

### Pour analyser l'évolution des capacités:
1. Charger le fichier `MCO_XXXXr.csv` pour chaque année
2. Filtrer sur le FINESS de Pitié (750100125)
3. Extraire les colonnes `LIT_*` et `PLA_*`

### Pour analyser les soins critiques:
1. Charger le fichier `REA_XXXXr.csv`
2. Noter qu'il y a **plusieurs lignes** par établissement (une par type d'unité UNI)
3. Agréger les lits par catégorie (Réa, SI, USC)

### Pour calculer des indicateurs:
- Taux d'occupation = Journées / (Lits × 365)
- DMS (Durée Moyenne de Séjour) = Journées / Séjours
- Taux de rotation = Séjours / Lits

In [10]:
print("\n📚 RESSOURCES COMPLÉMENTAIRES:")
print("="*50)
print("   • Documentation SAE: https://drees.solidarites-sante.gouv.fr/sources-outils-et-enquetes/00-la-statistique-annuelle-des-etablissements-sae")
print("   • Données ouvertes: https://data.drees.solidarites-sante.gouv.fr")
print("   • FINESS: https://finess.sante.gouv.fr")


📚 RESSOURCES COMPLÉMENTAIRES:
   • Documentation SAE: https://drees.solidarites-sante.gouv.fr/sources-outils-et-enquetes/00-la-statistique-annuelle-des-etablissements-sae
   • Données ouvertes: https://data.drees.solidarites-sante.gouv.fr
   • FINESS: https://finess.sante.gouv.fr
